<a href="https://colab.research.google.com/github/summermccune/Tokenization-Testing-for-Malware-Data/blob/main/Word2Vec/filteringAndEmbedding_UNIGRAMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "scipy==1.11.0"#change this line?
!pip install gensim
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install scikit-learn
!pip install glob
!pip install nltk
!pip install csv
!pip install collections
!pip install re
!pip install gc
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import nltk
from nltk.tokenize import word_tokenize
# from google.colab import drive
# drive.mount('/content/drive')
import csv
from collections import Counter
import re
import gc

In [2]:
def count(df, c):
  for row in df['Opcodes']:
     data = row.split()
     c.update(data)

In [4]:
def removeNonVocab(vocab, series):
  rows = []
  vocab_str = '|'.join(vocab)
  pattern = '\\b((?!\\b( |' + vocab_str + ')\\b).)*\\b'
  for row in series:
    row = re.sub(pattern, '', row)
    row = re.sub(' +', ' ', row)
    rows.append(row)
  return rows

In [6]:
#read in the data
FakeRean = pd.read_csv('FakeRean.csv')
OnLineGames = pd.read_csv('OnLineGames.csv')
Vobfus = pd.read_csv('Vobfus.csv')
Winwebsec = pd.read_csv('Winwebsec.csv')

FakeRean_df = pd.DataFrame(FakeRean)
OnLineGames_df = pd.DataFrame(OnLineGames)
Vobfus_df = pd.DataFrame(Vobfus)
Winwebsec_df = pd.DataFrame(Winwebsec)

#drop first column
FakeRean_df = FakeRean_df.iloc[:, 1:]
OnLineGames_df = OnLineGames_df.iloc[:, 1:]
Vobfus_df = Vobfus_df.iloc[:, 1:]
Winwebsec_df = Winwebsec_df.iloc[:, 1:]

In [7]:
#counting opcodes for data cleaning
countTotal = Counter()
count(FakeRean_df, countTotal)
count(OnLineGames_df,countTotal)
count(Vobfus_df,countTotal)
count(Winwebsec_df,countTotal)

In [8]:
#list for most common opcodes
total_count = countTotal.most_common(31)
countList = [x[0] for x in total_count]
print(countList)

['add', 'mov', 'push', 'pop', 'inc', 'xchg', 'call', 'or', 'dec', 'cmp', 'xor', 'sub', 'and', 'adc', 'sbb', 'lea', 'test', 'out', 'in', 'jmp', 'movl', 'int3', 'ret', 'imul', 'je', 'nop', 'lods', 'stos', 'scas', 'lret', 'jne']


In [9]:
#adding cleaned data to new dataframe
rows = []

#0 for FakeRean
FakeRean_rows = removeNonVocab(countList, FakeRean_df['Opcodes'])
for row in FakeRean_rows:
  row = row.split()
  rows.append((tuple(row), 0))

#1 for OnLineGames
OnLineGames_rows = removeNonVocab(countList, OnLineGames_df['Opcodes'])
for row in OnLineGames_rows:
  row = row.split()
  rows.append((tuple(row), 1))

#2 for Vobfus
Vobfus_rows = removeNonVocab(countList, Vobfus_df['Opcodes'])
for row in Vobfus_rows:
  row = row.split()
  rows.append((tuple(row), 2))

#3 for Winwebsec
Winwebsec_rows = removeNonVocab(countList, Winwebsec_df['Opcodes'])
for row in Winwebsec_rows:
  row = row.split()
  rows.append((tuple(row), 3))


In [10]:
#make csv of most common opcodes and their count
df = pd.DataFrame(total_count, columns=['Opcodes', 'Count'])
df.to_csv('MostCommonOpcodes.csv', index=False)

In [11]:
#turn rows tuple into df
df = pd.DataFrame(rows, columns=['Opcodes', 'MalwareType'])

#cleaned rows as csv
df.to_csv('UnigramFilteredOpcodes.csv', index=False)

In [ ]:
#word2vec N = 100, W = 30
model = Word2Vec(df['Opcodes'], min_count=1, vector_size=100, window=30)
embeddings = np.array([np.mean([model.wv[word] for word in text if word in model.wv], axis=0) for text in df['Opcodes']])
np.save('ALL_embeddings.npy', embeddings)

#word2vec N = 31, W = 10
model = Word2Vec(df['Opcodes'], min_count=1, vector_size=31, window=10)
embeddings = np.array([np.mean([model.wv[word] for word in text if word in model.wv], axis=0) for text in df['Opcodes']])
np.save('SVM_embeddings.npy', embeddings)

## Test Reading from the Created CSV

In [13]:
data = pd.read_csv('UnigramFilteredOpcodes.csv')
#turn to df
df = pd.DataFrame(data)
print(df.head())

                                             Opcodes  MalwareType
0  ('add', 'je', 'je', 'push', 'push', 'add', 'ad...            0
1  ('push', 'mov', 'push', 'push', 'mov', 'xor', ...            0
2  ('call', 'mov', 'pop', 'pop', 'pop', 'ret', 'i...            0
3  ('add', 'in', 'add', 'push', 'call', 'pop', 'r...            0
4  ('mov', 'push', 'push', 'push', 'mov', 'mov', ...            0


In [14]:
print(df.tail())

                                                Opcodes  MalwareType
3995  ('push', 'mov', 'sub', 'push', 'push', 'push',...            3
3996  ('push', 'mov', 'sub', 'push', 'push', 'push',...            3
3997  ('int3', 'int3', 'int3', 'int3', 'int3', 'int3...            3
3998  ('push', 'mov', 'mov', 'int3', 'int3', 'int3',...            3
3999  ('or', 'xchg', 'call', 'inc', 'mov', 'or', 'po...            3
